In [1]:
# Torch
import torch
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler
# Data Handling
import numpy as np
import pandas as pd
# System
import os
import pprint
import itertools
# Custom
import models
import datasets
from helpers import utils

In [7]:
########################## FILE SYSTEM SETTINGS ###########################

savedir_base = "/home/jovyan/work/runs/LCFCN"
datadir = "/home/jovyan/work/DENMARK/250x250"

####################### EXPERIMENT SETTINGS ################################

experiment_id = "d1efb632350537a1a2af171808903f4a"
run_path = os.path.join(savedir_base, experiment_id)
exp_dict = utils.loadJSON(os.path.join(run_path, 'exp_dict.json'))

pprint.pprint(exp_dict)

{'batch_size': 1,
 'dataset': {'name': 'denmark', 'transform': 'rgb_normalize'},
 'dataset_size': {'train': 'all', 'val': 'all'},
 'lr': 1e-05,
 'max_epoch': 5,
 'model': {'base': 'fcn8_vgg16', 'name': 'lcfcn'},
 'optimizer': 'adam'}


In [8]:
dataset_size = {'test': 10}
batch_size = 1

test_set = datasets.getDataset(dataset = exp_dict["dataset"]["name"],
                               split = "test",
                               datadir = datadir,
                               transform = exp_dict['dataset']['transform'],
                               dataset_size = dataset_size)

test_sampler = torch.utils.data.SequentialSampler(test_set)

test_loader = DataLoader(test_set,
                         sampler = test_sampler,
                         batch_size = batch_size,
                         num_workers = 2)

print("Testing Data loaded...")

In [9]:
n_classes = test_set.n_classes
model = models.getModel(exp_dict, n_classes).cuda()

run_model = utils.loadTorch(os.path.join(run_path, 'model_best.pth'))
model.loadStateDict(run_model)

print("Model loaded...")